In [109]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from database.scrape import get_all_stats
pd.set_option('display.max_columns', 70)

In [114]:
year = 2023
defense = 'mia'
URL = f"https://www.pro-football-reference.com/teams/{defense}/{year}/gamelog/"
res = requests.get(URL, verify=False)
soup = BS(res.content, "html.parser")
table = soup.find_all("table", {"id": f"gamelog_opp{year}"})

In [115]:
df = pd.read_html(str(table))[0]
flattened_columns = ["_".join(col).strip() for col in df.columns.values]
df.columns = flattened_columns
df.rename(
    columns={
        "Unnamed: 0_level_0_Rk": "GAME_ORDER",
        "Unnamed: 1_level_0_Year": "YEAR",
        "Unnamed: 2_level_0_Date": "DATE",
        "Unnamed: 3_level_0_G#": "GAME_NUMBER",
        "Unnamed: 0_level_0_Week": "WEEK",
        "Unnamed: 5_level_0_Age": "AGE",
        "Unnamed: 6_level_0_Tm": "TEAM",
        "Unnamed: 8_level_0_Opp": "OPPONENT",
        "Unnamed: 9_level_0_Result": "RESULT",
    },
    inplace=True,
)
# df.drop(columns=["Unnamed: 7_level_0_Unnamed: 7_level_1"], inplace=True)
df.columns = df.columns.str.upper()
df.columns = df.columns.str.replace(".", "")
df.columns = df.columns.str.replace("/", "_")
df.columns = df.columns.str.replace(".", "")
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("%", "_PCT")

# df.fillna(0, inplace=True)
df['YEAR'] = year
df['DEF_TEAM'] = defense.upper()
df_def = df[[ 'DEF_TEAM', 'YEAR', 'WEEK', 'PASSING_CMP', 'PASSING_ATT', 'PASSING_YDS',
       'PASSING_TD', 'PASSING_INT', 'PASSING_SK', 'PASSING_Y_A', 'PASSING_NY_A', 'PASSING_CMP_PCT', 'PASSING_RATE',
       'RUSHING_ATT', 'RUSHING_YDS', 'RUSHING_Y_A', 'RUSHING_TD',]]
df_def.dropna(inplace=True)

C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_11488\2176474648.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def.dropna(inplace=True)


In [116]:
rolling = df_def[['PASSING_CMP', 'PASSING_ATT', 'PASSING_YDS',
       'PASSING_TD', 'PASSING_INT', 'PASSING_SK', 'PASSING_Y_A',
       'PASSING_NY_A', 'PASSING_CMP_PCT', 'PASSING_RATE', 'RUSHING_ATT',
       'RUSHING_YDS', 'RUSHING_Y_A', 'RUSHING_TD']].rolling(window=15, min_periods=1).mean()

df_dfns = df_def[['DEF_TEAM', 'YEAR', 'WEEK']].join(rolling, how='left')

In [117]:
#for future projections
while len(df_dfns) < 17:

    new_row = df_dfns.iloc[-1].copy()

    # Increment the "week" column in the new row
    new_row['WEEK'] += 1

    # Append the new row to the DataFrame
    df_dfns = df_dfns.append(new_row, ignore_index=True)

df_dfns

,DEF_TEAM,YEAR,WEEK,PASSING_CMP,PASSING_ATT,PASSING_YDS,PASSING_TD,PASSING_INT,PASSING_SK,PASSING_Y_A,PASSING_NY_A,PASSING_CMP_PCT,PASSING_RATE,RUSHING_ATT,RUSHING_YDS,RUSHING_Y_A,RUSHING_TD
0,MIA,2023,1,23.000000,33.000000,200.000000,1.000000,0.000000,3.000000,6.900000,5.600000,69.700000,95.500000,40.000000,233.000000,5.800000,3.000000
1,MIA,2023,2,27.000000,37.500000,200.000000,1.000000,0.500000,3.500000,6.200000,4.950000,71.750000,88.450000,32.500000,160.500000,4.650000,2.000000
2,MIA,2023,3,25.666667,37.666667,231.333333,1.000000,0.666667,2.666667,6.833333,5.800000,68.000000,86.500000,28.333333,130.000000,4.266667,1.333333
3,MIA,2023,4,24.500000,34.500000,251.000000,1.750000,0.500000,2.500000,8.325000,7.225000,72.000000,104.350000,28.500000,123.500000,4.100000,1.500000
4,MIA,2023,5,24.200000,34.000000,237.400000,1.400000,0.400000,3.400000,7.940000,6.720000,71.980000,100.640000,28.600000,115.800000,3.860000,1.200000
5,MIA,2023,6,24.166667,34.833333,229.166667,1.333333,0.333333,3.500000,7.566667,6.333333,70.233333,97.533333,28.000000,114.500000,3.933333,1.166667
6,MIA,2023,7,24.000000,34.285714,233.000000,1.428571,0.428571,3.428571,7.771429,6.500000,70.800000,98.800000,28.857143,112.285714,3.785714,1.285714
7,MIA,2023,8,23.375000,33.625000,221.500000,1.500000,0.500000,3.375000,7.500000,6.237500,70.137500,97.150000,27.625000,107.875000,3.825000,1.125000
8,MIA,2023,9,23.000000,33.222222,216.222222,1.555556,0.444444,3.222222,7.355556,6.144444,69.755556,97.911111,27.222222,106.222222,3.833333,1.000000
9,MIA,2023,11,23.100000,34.000000,220.600000,1.500000,0.700000,3.100000,7.280000,6.130000,68.630000,93.610000,26.100000,99.200000,3.680000,0.900000
